In [7]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
import folium
from bs4 import BeautifulSoup
import lxml
print('Libraries imported.')

Libraries imported.


<h4>Download data and parse it</h4>

In [48]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(r.text, 'html.parser')
table=soup.find('table', attrs={'class':'wikitable sortable'})

<h4>Get headers</h>

In [49]:
headers=table.findAll('th')
for i, head in enumerate(headers): headers[i]=str(headers[i]).replace("<th>","").replace("</th>","").replace("\n","")

<h4>Find all items and skip first one</h4>

In [50]:
rows=table.findAll('tr')
rows=rows[1:len(rows)]

<h4>Skip all meta symbols and line feeds between rows:</h4>

In [51]:
for i, row in enumerate(rows): rows[i] = str(rows[i]).replace("\n</td></tr>","").replace("<tr>\n<td>","")

<h4>Make dataframe, expand rows and drop the old one:</h4>

In [52]:
df=pd.DataFrame(rows)
df[headers] = df[0].str.split("</td>\n<td>", n = 2, expand = True) 
df.drop(columns=[0],inplace=True)

<h4>Skip not assigned boroughs</h4>

In [53]:
df = df.drop(df[(df.Borough == "Not assigned")].index)

<h4>Give "Not assigned" Neighborhoods same name as Borough</h4>

In [54]:
df.Neighbourhood.replace("Not assigned", df.Borough, inplace=True)

<h4>Copy Borough value to Neighborhood if NaN</h4>

In [55]:
df.Neighbourhood.fillna(df.Borough, inplace=True)

<h4>Drop duplicate rows</h4>

In [56]:
df=df.drop_duplicates()

<h4>Extract titles from columns</h4>

In [57]:
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

df.update(
    df.Borough.loc[
       lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

<h4>Delete Toronto annotation from Neighbourhood</h4>

In [58]:
df.update(
    df.Neighbourhood.loc[
       lambda x: x.str.contains('Toronto')
   ].str.replace(", Toronto",""))
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace("\(Toronto\)",""))

<h4>Combine multiple neighborhoods with the same post code</h4>

In [59]:
df2 = pd.DataFrame({'Postcode':df.Postcode.unique()})
df2['Borough']=pd.DataFrame(list(set(df['Borough'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=pd.Series(list(set(df['Neighbourhood'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=df2['Neighborhood'].apply(lambda x: ', '.join(x))
df2.dtypes

df2.head(12)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


<h4>Print the number of rows of your dataframe</h4>

In [61]:
df2.shape

(103, 3)